In [5]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil

2025-05-14 17:11:48.006214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747257108.020019   37483 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747257108.024152   37483 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747257108.034724   37483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747257108.034738   37483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747257108.034739   37483 computation_placer.cc:177] computation placer alr

In [2]:
general_directory ='/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'

subdirectories = ['/Geometry','/Magnitude','/Pression','/U001','/U002']
geo_path = general_directory + subdirectories[0]
v_path =  general_directory + subdirectories[1]
p_path =  general_directory + subdirectories[2]
vx_path = general_directory + subdirectories[3]
vy_path = general_directory + subdirectories[4]

In [2]:
# Take a sample
n_sample = 8000   # /20 000

# split dataset 
n_train = 0.7
n_valid = 0.2
n_test =  0.1

# Resizing 
w,h=256,128


In [4]:
#method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

# method for turning to a grey or binary image 
def processing(image,option):
        x = get_img(image)  
        x = rgb2gray(x)       # It returns a grayscale image with floating point values in the range from 0 to 1
        x =cv.resize(x,(w,h)) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            x=ski.util.img_as_ubyte(x)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(x)) #  Otsu’s method calculates an “optimal” threshold

            _,x= cv.threshold(x, best_value_threshold, 255, cv.THRESH_BINARY)
            x=x/255.
        x=x[:, :, np.newaxis]
        return x

In [5]:
# method for creating npy of each image
def npy_create (path,name,option,sample=n_sample):
    images= sorted([os.path.join(path,fname) for fname in os.listdir(path) if fname.endswith(".png")])
    if n_sample < 20000:
        I = images[1:sample+1]
    name_file = name
    new_file= general_directory + '/' + name_file
    os.makedirs(new_file, exist_ok=True)
    for img in I:
        name_img = os.path.basename(img)
        path=new_file+'/'+ name_img
        x=processing(img,option)
        np.save(path,x)

In [3]:
# Mask each image
def mask (name_file,name_data,path_file_1,path_file_2=None):
    print('enter')
    if path_file_2 == None:
        path_1 = Path(path_file_1)
        file_npy_1=sorted(list(path_1.glob('*.npy')))
        current_directory = Path.cwd()
        save_in = str(current_directory) + '/' + name_file 
        os.makedirs(save_in, exist_ok=True)
        n = 1
        for f1 in file_npy_1:
            array_1 = np.load(f1)
            save= save_in+ '/'+name_data+f'_{n:05d}.png'
            array_2d = img_as_ubyte(array_1.squeeze())
            io.imsave(save, array_2d)
            n=n+1
    else:
        path_1 = Path(path_file_1)
        path_2 = Path(path_file_2)
        file_npy_1=sorted(list(path_1.glob('*.npy')))
        file_npy_2=sorted(list(path_2.glob('*.npy')))
        current_directory = Path.cwd()
        save_in = str(current_directory) + '/' + name_file 
        os.makedirs(save_in, exist_ok=True)
        n = 1 
        for f1, f2 in zip(file_npy_1,file_npy_2):
            array_1 = np.load(f1)
            array_2 = np.load(f2)
            array_masked=array_1*array_2
            save= save_in+ '/'+name_data+f'_{n:05d}.png'
            array_2d = img_as_ubyte(array_masked.squeeze())
            io.imsave(save, array_2d)
            n=n+1


def split_data(file,n_train=n_train,n_valid=n_valid,n_test=n_test):
     n_sample=len(file)
     if (round(n_train + n_valid + n_test,2) == 1.0):
        n_train= math.floor(n_sample*n_train)
        n_valid = math.floor(n_sample*n_valid) + n_train
        n_test = math.floor(n_sample*n_test) + n_valid
        (train,valid,test) = (file[0:n_train], file[n_train:n_valid],file[n_valid:n_test]) 
     return train,valid,test
    
# separating images into different folders
def create_files(name_file,general_directory = '/home/ppgi/Trabajo/predicting-flow-patterns'): 
    
   # name_file = geo_masked mag_masked  vx_masked vy_masked p_masked
   # path = /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/mag_masked
    
    path=general_directory + name_file
    file = Path(path)
    
    images= sorted(file.glob('*.png')) 
    train,valid,test =  split_data(images)
    
    path_train = general_directory +name_file+ '/train'
    path_test = general_directory + name_file+'/test'
    path_valid = general_directory +name_file+ '/valid'
    print('creating')
    os.makedirs(path_train, exist_ok=True)
    os.makedirs(path_test, exist_ok=True)
    os.makedirs(path_valid, exist_ok=True)
    
    for img in train:
        p =  path_train + '/' + img.name
        shutil.move(img,p)
    
    for img in valid:  
        p = path_valid + '/' + img.name
        shutil.move(img,p)

    for img in test: 
        p = path_test + '/' + img.name
        shutil.move(img,p)            

In [7]:
# ***** 1 *******
npy_create (geo_path,'geo_npy',True)
npy_create (v_path,'v_npy',False)
npy_create (p_path,'p_npy',False)
npy_create (vx_path,'vx_npy',False)
npy_create (vy_path,'vy_npy',False)


In [8]:
# ******** 2 *************
path_file_1 = general_directory + '/geo_npy'
path_file_2 = general_directory +'/v_npy'
path_file_3 = general_directory +'/vx_npy'
path_file_4 = general_directory +'/vy_npy'
path_file_5 = general_directory +'/p_npy'


#path_file_1='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI/geo_npy'
#path_file_2='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI/v_npy'
#path_file_3='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI/vx_npy'
#path_file_4='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI/vy_npy'
#path_file_5='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI/p_npy'

mask('geo_masked','g',path_file_1)
mask('mag_masked','v',path_file_1,path_file_2)
mask('vx_masked','vx',path_file_1,path_file_3)
mask('vy_masked','vy',path_file_1,path_file_4)
mask('p_masked','p',path_file_1,path_file_5)

enter


/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00007.png is a low contrast image
  io.imsave(save, array_2d)
/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00012.png is a low contrast image
  io.imsave(save, array_2d)
/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00025.png is a low contrast image
  io.imsave(save, array_2d)
/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00029.png is a low contrast image
  io.imsave(save, array_2d)
/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00033.png is a low contrast image
  io.imsave(save, array_2d)
/tmp/ipykernel_35394/4184920992.py:15: UserWarning: /home/ppgi/Trabajo/predicting-flow-patterns/geo_masked/g_00036.png is a low contrast image
  

enter
enter
enter
enter


In [6]:
create_files('/geo_masked')
create_files('/mag_masked')
create_files('/vx_masked')
create_files('/vy_masked')
create_files('/p_masked')

creating
creating
creating
creating
creating
